# 平均報酬強化学習

参考：
* [Average-reward model-free reinforcement learning: a systematic review and literature mapping](https://arxiv.org/abs/2010.08920)
* [Markov Decision Processes: Discrete Stochastic Dynamic Programming](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470316887)
* [Average reward reinforcement learning: Foundations, Algorithms, and Empirical Results](https://link.springer.com/content/pdf/10.1007/BF00114727.pdf)
* [Examining average and discounted reward optimality  criteria in reinforcement learning](https://arxiv.org/abs/2107.01348): 結構良かった。おすすめ。

## マルコフ連鎖の用語

参考：
* [Average reward reinforcement learning: Foundations, Algorithms, and Empirical Results](https://link.springer.com/content/pdf/10.1007/BF00114727.pdf)
* [Markov Decision Processes: Discrete Stochastic Dynamic Programming](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470316887)

**表記**
* $\left\{X_n, n=0,1,2, \ldots\right\}$を離散状態空間$S$に値を取る確率変数とします．$S$は有限もしくは可算です．
* 次を$n \geq 1$かつ$j_k \in S, 0 \leq k \leq n$について満たすとき，$\left\{X_n, n=0,1,2, \ldots\right\}$は**マルコフ**であると言います：$P\left\{X_n=j_n \mid X_{n-1}=j_{n-1}, X_{n-2}=j_{n-2}, \ldots, X_0=j_0\right\}=P\left\{X_n=j_n \mid X_{n-1}=j_{n-1}\right\}$
* $p^m(j\mid s) = P\left\{X_{n+m}=j \mid X_n=s\right\}$を，$m$ステップ後に$s$から$j$に遷移する確率とします．
  * 簡単な計算で示せますが，これは遷移確率行列の$m$乗の$P^m$の要素になります．これを[チャップマン–コルモゴロフ方程式](https://manabitimes.jp/math/1060#4)と言います．
* $S$上の実数関数$g(\cdot)$について，$E_s\left\{g\left(X_n\right)\right\} \equiv \sum_{j \in S} g(j) p^n(j \mid s)$を，$X_0=s$から始まって$n$ステップ目の確率変数の期待値とします．
* また，$P_s\left\{X_n \in A\right\} \equiv E_s\left(I_{(A)}\left(X_n\right)\right\}=\sum_{j \in A} p^n(j \mid s)$の表記も導入します．



### 状態についての用語

#### transientとrecurrent （訪問回数について）

$s \in S$について，次の変数を導入します：
* $\nu_s$：$s$を訪問する回数（無限時間かつ$s$からスタートして？）
* $\tau_s$：$s$からスタートして，$s$を初めて訪問する時刻

<!-- ここで，$E_s\{\tau_s\} < \infty$のとき，有限ステップ以内に$s$を必ず再訪問することがわかります．そうでないと，期待値を取ると$\infty$になってしまいます．
また，$P_s\left(\tau_s<\infty\right)<1$は$P_s\left(\tau_s=\infty\right)>0$を意味するので，$s$を再訪問しない確率が少しあることがわかります． -->

この変数に対して，次の用語を導入します：
1. $E_s\{\tau_s\} < \infty$（再訪問するステップの期待値が有限）かつ$P_s\left(\tau_s<\infty\right)<1$（$s$を**永遠にに再訪問しない**確率が少しある）：これは矛盾するので，ありえません．
2. $E_s\{\tau_s\} = \infty$（再訪問するステップの期待値が無限）かつ$P_s\left(\tau_s<\infty\right)<1$（$s$を**永遠に再訪問しない**確率が少しある）：このような$s$を**transient**である，と呼びます．
3. $E_s\{\tau_s\} < \infty$（再訪問するステップの期待値が有限）かつ$P_s\left(\tau_s<\infty\right)=1$（$s$に有限ステップで必ず再訪問する）：このような$s$を**positive recurrent**である，と呼びます
4. $E_s\{\tau_s\} = \infty$（再訪問するステップの期待値が無限）かつ$P_s\left(\tau_s<\infty\right)=1$（$s$に有限ステップで必ず再訪問する）：このような$s$を**null recurrent**である，と呼びます．

次のMDPがNull stateの例です（Birth-death chainと呼ばれたりします．[参考](https://www2.isye.gatech.edu/~spyros/courses/IE7201/Spring-22/Discrete-Time%20Birth-Death%20Process.pdf)）

![birth-death](figs/birth-death-chain.png)

一番左の状態$0$に注目しましょう．次の３パターンがありえます．
1. $p < 1/ 2$のとき，右に行く確率が高くなります．よって，$0$を再訪問するステップの期待値が無限かつ再訪問しない確率が少しあるので，これはTransientです．
2. $p > 1 / 2$のとき，左に行く確率が高くなります．よって，$0$を再訪問するステップの期待値が有限かつ$s$に必ず再訪問するので，これはpositive recurrentです
3. $p=1/2$のとき，$0$を再訪問する確率はありますが，一方で，再訪問する期待値は$\infty$です．よってnull recurrentです．

null recurrentとpositive recurrentは区別しないことが多いです（どちらもまとめてrecurrentと呼ぶ）．
また，recurrentな状態は**ergodic**とも呼びます．
次はrecurrentとtransientの必要十分条件です．

* recurrent : $E_s\left\{\nu_s\right\}=\sum_{n=0}^{\infty} p^n(s \mid s)=\infty$
* transient : $E_s\left\{\nu_s\right\}=\sum_{n=0}^{\infty} p^n(s \mid s)<\infty$


#### communicateとirreducible （他の状態との関係）

**２つの状態の関係**
* $p^n(j \mid s)>0$を満たす$n \geq 0$が存在するとき，$j$は$s$から**accessible**である，と言い，$s\to j$と書きます
  * そうでない場合は**inaccessible**と書きます
* $s \to j$かつ$j \to s$の場合，$s$と$j$は**communicate**である，と言います（つまりどちらからでも到達可能）


**複数の状態の関係**
* $C\subseteq S$は，全ての$s \notin C$が$j \in C$から**accessible**ではない場合，$C$はclosedである，と言います．つまり，**$C$から外に出ていけない場合です．**
* また，$C$がclosedであり，かつ$C$のプロパーな部分集合（$C$を含まない部分集合のこと）の全てがclosedではないとき，$C$を**irreducible**と言います．
  * これは$C$が**全連結**なグラフであることと同じです（ちょっと考えればこれが必要十分条件であることはわかる）．
  * つまり，$C$から外に出られない＆$C$の中にClosedな集合がない場合です．「これ以上reduceできない」って意味です．
* irreducibleかつclosedな集合であり，一つの状態からなるものを，**absorbing**である，と言います．
* Closedかつirreducibleな集合は，しばしば異なるマルコフ連鎖とみなされます．


#### 周期

* $p^n(s\mid s) > 0$であるような$n$の最大公約数を$s$の**period**と言います．つまり，$s$からスタートして$s$に戻ってくるのが$T$の倍数に限るとき，$T$のことを連鎖の周期と言います．
* $p(s \mid s) > 0$であるような$s$を**aperiodic**である，と言います．例えばどのような経路を使っても変えれる場合はaperiodicになります．[RL_AverageReward_Basic_fundamental_theorem_proof.ipynb](RL_AverageReward_Basic_fundamental_theorem_proof.ipynb)参照．
* **Periodは状態の集合ついての性質です．** 実際，Closedかつirreducibleな集合の状態は全て同じperiodを持ちます．[RL_AverageReward_Basic_fundamental_theorem_proof.ipynb](RL_AverageReward_Basic_fundamental_theorem_proof.ipynb)参照．
  * もしperiodが1より大きければ，その集合はperiodicと言います
  * periodが1の場合，その集合はaperiodicと言います
* irreducibleなマルコフ連鎖のperiodが1より大きい場合，periodicな連鎖と言います．同様に，1の場合はaperiodicな連鎖，と言います．



#### 遷移確率の標準形

recurrentな状態の集合は，互いに素なclosed irreducible集合$C_k$に分解することができます（これは多分深さ優先探索で[強連結成分分解](https://manabitimes.jp/math/1250)するのと同じ）．
ここで，$k=1, 2, \dots, m$であり，$S$が有限ならば$m$も有限です．このとき，
$$
S=C_1 \cup C_2 \cup \cdots \cup C_m \cup T
$$
と書けます．ここで$T$はどの$C_k$にも属さないtransientな状態集合です．
このとき，遷移確率行列は
$$
P=\left[\begin{array}{cccccc}
P_1 & 0 & 0 & . & . & 0 \\
0 & P_2 & 0 & . & . & 0 \\
. & & . & & & \\
. & & & . & & \\
0 & & & & P_m & 0 \\
Q_1 & Q_2 & \cdot & \cdot & Q_m & Q_{m+1}
\end{array}\right]
$$
として表現できます．これを遷移確率の標準形と呼びます．

ここで，$P_i$は$C_i$内の状態遷移を表し，$Q_i$は$T$から$C_i$への状態遷移を表します．
また，$Q_{m+1}$は$T$内の状態遷移です．

---

**補足**

次のアルゴリズムはマルコフ連鎖において，
* closedかつirreducibleな集合
* transientな状態集合

を見つけることができます．
次の性質を使います：

1. 状態$s$は任意の状態$j \neq s$について$p(s \mid s)>0$ かつ $p(j \mid s)=0$ ならば，absorbing
2. 状態$s$がabsorbingかつ$p(s\mid k) > 0$ならば，$k$はtransient
3. 状態$s$がtransientかつ$p(s\mid k) > 0$ならば，$k$はtransient
4. 状態$i$が$j$とcommunicatesであり，$j$がtransientならば，$i$はtransient．$j$がrecurrentならば$i$もrecurrent．

この性質は「行列$P$について，どこに$0$があり，どこに非$0$があるか？」にしか依存してないことに注意しましょう．よって，$P$の分解のみが重要ならば，$P$は

* $b(j \mid i)=1$ if $p(j \mid i)>0$, and 0 otherwise.

な行列$B$とみなしても問題ありません．

---

さて，状態空間$S=\{1, 2, \dots, N\}$を考えましょう．
以下のアルゴリズムでは

* $L(i)$は状態$i$に割り当てられたラベルを表します
    * recurrent: R 
    * transient: T
    * unlabeled: O

とします．
* $U$をunlabeledな状態の集合
* $S(i)$を「$i$とcommunicatingだと判断された状態の集合」とします．
* $W$をステップ5で構築された行列の行と列のインデックスの集合

とします．

1. 初期化：$S(i)=\{i\}$, $L(i)=O$，$U=S$，$W=S$
2. ... (TODO: 続き)



## マルコフ連鎖の種類

マルコフ連鎖はいくつかのクラスに分類できます

1. **irreducible chain**: 単一のclosedな状態集合からなるとき，マルコフ連鎖をirreducibleである，と言います（Transientな集合は存在しません）．
2. **Unichain**: 有限な状態集合について，一つのclosedな状態集合と一つ（もしくは空の）Transientな集合からなるとき，マルコフ連鎖をUnichainである，と言います．
3. **Multichain** そうでない場合はMultichainである，と言います．


### マルコフ決定過程の用語

上でやったのは**マルコフ連鎖**の用語です．
こっちはMDPの用語ですね．

#### 状態集合についての用語

**コメント**: 非周期性がなくても定常分布が存在するので，非周期性がなくてもergodicって呼ぶ場合があります．青本参照．

1. 次を満たす状態の集合$X$は，$\pi$において**ergodic**もしくは**recurrent**である，と言います．
    * $X$の中の全ての状態はrecurrentであり，また，お互いにcommunicateである．
2. もし全ての状態によってergodicな状態集合が形成されれば，そのマルコフ連鎖は**irreducible**（既約）である，と言います．

#### MDPの種類

1. **ergodic**：次を満たすとき，そのMDPは**ergodic**もしくは**recurrent**である，と言います．
    * **任意の**（決定的かつ定常な）方策が，単一のrecurrentな状態集合をもつ
2. **Unichain**：次を満たすとき，そのMDPは**unichain**である，と言います．
    * **任意の**方策が，単一のrecurrentな状態集合と(空でも良い)transientな状態集合をもつ
3. **Communicating**：次を満たすとき，そのMDPは**communicating**である，と言います．
    * 全ての状態のペアがcommunicateになるような定常な方策が存在する
4. **Weakly Communicating**：次を満たすとき，そのMDPは**weakly communicating**である，と言います．
    * 状態空間が次の２つに分割できる：
        1. すべての状態が，任意の定常方策について，transientである．
        2. 任意の２つの状態を行き来できる定常方策が存在する．
3. **Multichain**：次を満たすとき，そのMDPは**multichain**である，と言います．
    * ２つ以上のrecurrentな状態集合をもつような定常な方策が存在する．
    * Multichainはcommunicatingな場合もありますが，そうでない場合もあります．教科書Example 8.3.1参照．

こんな感じの関係になります（上に行くほど一般的）．

![](figs/MDP-relationship.png)

---

recurrentやtransientな状態の例については論文のFigure 2などを参照してください．
